In [99]:
import pandas as pd
import bs4

In [100]:
#загружаем тестовую выборку
with open('test_kaggle.csv', 'r') as f:
    data2 = f.read()
columns = ['mes']
test_review=pd.DataFrame(columns=columns)
for r in bs4.BeautifulSoup(data2, 'lxml').find_all('review'):
    a=r.text
    new_line = {'mes':a}
    test_review=test_review.append(new_line, ignore_index=True)

In [101]:
#загружаем обучающую выборку (отзывы про телефоны с сайта torg.mail.ru)
data_rew=pd.read_csv('review_hw66.txt', sep='\t')

In [102]:
data_rew.head()

,Unnamed: 0,mes,rating
0,0,"Купил аппарат в ДНС. На вид крепкий, водонепро...",0
1,1,Мой третий LG.Честно не повезло. Грешу на сбор...,0
2,2,"Аппарат, из-за которого решил больше не покупа...",0
3,3,при захлопывании задней крышки телефон зависае...,0
4,4,"Месяц разочарования пользования этим смартом, ...",0


In [103]:
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn import naive_bayes
from sklearn.grid_search import GridSearchCV

In [104]:
data_rew.groupby('rating').describe()

Unnamed: 0                                                      \
            count          mean           std    min      25%     50%   
rating                                                                  
0          3010.0  10971.834219  15619.985793    0.0  1892.25  3133.0   
1          5430.0  18438.990792  23323.933408  717.0  4228.50  8149.5   

                           
             75%      max  
rating                     
0       16178.00  93313.0  
1       21491.75  97437.0

In [105]:
#добавим еще плохих
a1=data_rew[data_rew.rating==1]
a0=data_rew[data_rew.rating==0]
a00=data_rew[data_rew.rating==0][:2000]

frames1 = [a0, a1,a00]
a0a1 = pd.concat(frames1).reset_index(drop=True)

In [106]:
#обучающий и тестовый набор
msg_train = a0a1['mes']
msg_test = test_review['mes']
label_train = a0a1['rating']

print len(msg_train), len(msg_test), len(msg_train) + len(msg_test)

10440 100 10540


In [107]:
#собираем пайплайн
pipeline = Pipeline([
    ('bow', CountVectorizer(max_df=0.8,min_df=3)), 
    ('tfidf', TfidfTransformer()),
    ('classifier', naive_bayes.BernoulliNB()),
])

In [108]:
#выбираем параметры, которые будем перебирать 
param = {
    'tfidf__norm':('l1', 'l2', None),
    'tfidf__use_idf': (True, False),
    'bow__ngram_range': [ (1, 2),(1, 3)],
            }

grid_BernoulliNB = GridSearchCV(
    pipeline,  
    param_grid=param,  
    refit=True,  #сразу подставим лучший вариант
    scoring='accuracy',  
    cv=StratifiedKFold(label_train, n_folds=5),  
)

In [109]:
#ищем лучшую комбинацию
%time BernoulliNB_detector = grid_BernoulliNB.fit(list(msg_train), list(label_train))

Wall time: 2min 31s


In [110]:
#предсказываем для тестовой выборки заказчика
test_predict=BernoulliNB_detector.predict(test_review['mes'])

In [111]:
#танцы с бубном, чтобы соответствовать формату
df_file = pd.DataFrame(test_predict, index=test_review.index, columns=['y'])

In [112]:
df_file.index.names = ['id']

In [113]:
i=0
while i <100:
    if df_file['y'][i]==0:
        df_file.loc[i,'y'] = 'neg'
    else: df_file.loc[i,'y'] = 'pos'
    i=i+1

In [114]:
#пишем в файл
df_file.to_csv('66.csv')